## Задача №1

Предположим, у нас есть набор признаков, которые вычисляются независимо от эксперимента. Используя эти признаки, нужно разбить объекты на страты так, чтобы дисперсия стратифицированного среднего была минимальна и доля каждой страты была не менее 5% от всех данных.

Данные разбиты на 2 части. Первая часть доступна для исследования по ссылке stratification_task_data_public.csv. Решение будет проверяться на второй части данных. Значения в столбцах
x1, ..., x10 — признаки, которые можно использовать для вычисления страт. Значения в столбце y — измерения, по которым будет вычисляться целевая метрика эксперимента.

Подходы формирования страт можно посмотреть в блокноте "Стратификация_задача_1.ipynb"

In [11]:
import pandas as pd
import numpy as np
from scipy import stats


In [2]:
train_strats = pd.read_csv('stratification_task_data_public.csv')
train_df = train_strats.copy()


In [3]:
# Вычисляет стратифицированную дисперсию и минимальную долю страт
def calc_strat_params(df):
    # дисперсия в каждой страте
    strat_vars = df.groupby('strat')['y'].var()
    
    strat_vars_tmp = strat_vars.copy().reset_index()
    strat_vars_tmp = strat_vars_tmp.rename({
        'y': 'variance'
    }, axis=1)
    
    # вес каждой страты
    weights = df['strat'].value_counts(normalize=True)
    weights_tmp = weights.copy().reset_index()
    weights_tmp = weights_tmp.rename({
        'strat': 'weight',
        'index': 'strat'
    }, axis=1)
    
    strat_vars_tmp = strat_vars_tmp.merge(weights_tmp, how='inner', on='strat')#[['strat', '']]
    print('дисперсия и веса по стратам')
    print(strat_vars_tmp, end='\n\n')
    
    # стратифицированная дисперсия 
    stratified_var = (strat_vars * weights).sum()
    
    # минимальная доля страты
    min_part = df['strat'].value_counts(normalize=True).min()
    
    return stratified_var, min_part

# Выводит стратифицированную дисперсию и минимальную долю страт
def print_strat_params(df):
    stratified_var, min_part = calc_strat_params(df)
    print('stratified_var =', stratified_var, ', min share =', str(round(min_part*100, 2))+'%')

In [4]:
train_strats['strat'] = 0
print('дисперсия без стратификации:')
print_strat_params(train_strats)

дисперсия без стратификации:
дисперсия и веса по стратам
   strat      variance  weight
0      0  66078.094333     1.0

stratified_var = 66078.09433343346 , min share = 100.0%


In [5]:
# формирует страты для входящего датафрейма
def build_strats(df):
    uniq_strats = list(df.x7.unique())
    
    group_1 = range(1988, 1998)
    for strat_item in group_1:
        df.loc[df['x7'] == strat_item, 'strat'] = strat_item
        uniq_strats.remove(strat_item)
        
    group_2 = [1998, 1981]
    for strat_item in group_2:
        df.loc[df['x7'] == strat_item, 'strat'] = group_2[0]
        uniq_strats.remove(strat_item)
        
    group_3 = [x for x in uniq_strats if ((x <= 1980) or (x >= 2000))]
#     print('3 -> ', group_3)
    for strat_item in group_3:
        df.loc[df['x7'] == strat_item, 'strat'] = group_3[0]
        uniq_strats.remove(strat_item)
        
    group_4 = [x for x in uniq_strats]
#     print('4 -> ', group_4)
    for strat_item in group_4:
        df.loc[df['x7'] == strat_item, 'strat'] = group_4[0]
        uniq_strats.remove(strat_item)
           
    print_strat_params(df)

In [6]:
print('дисперсия со стратификацией:')
build_strats(train_df)

дисперсия со стратификацией:
дисперсия и веса по стратам
     strat      variance  weight
0   1987.0  50748.436720  0.1623
1   1988.0  43550.290310  0.0508
2   1989.0  48230.109476  0.0635
3   1990.0  47059.210799  0.0679
4   1991.0  48991.872146  0.0835
5   1992.0  50323.906093  0.0864
6   1993.0  47963.510437  0.0847
7   1994.0  49602.361371  0.0838
8   1995.0  42475.998720  0.0771
9   1996.0  48382.584194  0.0672
10  1997.0  54927.887730  0.0550
11  1998.0  54281.791778  0.0506
12  2003.0  61324.289892  0.0672

stratified_var = 49779.64634901167 , min share = 5.06%


In [7]:
import pandas as pd
import numpy as np


def get_strats(df_features):
    """Возвращает страты объектов.

    :param df_features (pd.DataFrame): таблица с признаками x1,...,x10
    :return (list | np.array | pd.Series): список страт объектов размера len(df).
    """
    
    df_features['strat'] = -1
    uniq_strats = list(df_features.x7.unique())
    
    group_1 = range(1988, 1998)
    for strat_item in group_1:
        df_features.loc[df_features['x7'] == strat_item, 'strat'] = strat_item
        uniq_strats.remove(strat_item)
        
    group_2 = [1998, 1981]
    for strat_item in group_2:
        df_features.loc[df_features['x7'] == strat_item, 'strat'] = group_2[0]
        uniq_strats.remove(strat_item)
        
    group_3 = [x for x in uniq_strats if ((x <= 1980) or (x >= 2000))]
#     print('3 -> ', group_3)
    for strat_item in group_3:
        df_features.loc[df_features['x7'] == strat_item, 'strat'] = group_3[0]
        uniq_strats.remove(strat_item)
        
    group_4 = [x for x in uniq_strats]
#     print('4 -> ', group_4)
    for strat_item in group_4:
        df_features.loc[df_features['x7'] == strat_item, 'strat'] = group_4[0]
        uniq_strats.remove(strat_item)
        
    return df_features['strat']

## задача № 2: Функция стратифицированного распределения клиентов по группам

Допустим, мы заранее определили множество клиентов, которые будут участвовать в эксперименте. Их страты известны. Нужно написать функцию, которая будет стратифицировано распределять их по группам.

Распределение по группам будем считать стратифицированным, если для каждой страты количество клиентов этой страты в группах отличаются не более, чем на 1.

Реализуйте функцию split_stratified

In [4]:
import numpy as np
import pandas as pd

def split_stratified(strats):
    """Распределяет объекты по группам (контрольная и экспериментальная).

    :param strats (np.array): массив с разбиением на страты.
    :return groups (np.array): массив из 0 и 1,
        0 - контрольная группа, 1 - экспериментальная.
    """
    
    strats_dict = {}
    groups = np.zeros(len(strats))
    
    for item in np.unique(strats).tolist():
        strats_dict[item] = int(np.count_nonzero(strats == item) / 2)
        
    for i in range(len(strats)):
        element = strats[i]
        if strats_dict[element] > 0:
            groups[i] = 1
            strats_dict[element] -= 1
            
    return groups.astype(int)
    

In [5]:
df = pd.DataFrame({'strat': [1, 2, 2, 2, 1, 1, 1, 3, 3]})
df['group'] = split_stratified(df['strat'].values)
# df = pd.DataFrame({
#   'strat': [1, 2, 2, 2, 1, 1, 1, 3, 3],
#   'group': [1, 0, 0, 1, 0, 0, 1, 0, 1]
# })

In [6]:
print(df)

   strat  group
0      1      1
1      2      1
2      2      0
3      2      0
4      1      1
5      1      0
6      1      0
7      3      1
8      3      0


## Задача 3: Функция для оценки эксперимента с применением постстратификации

Реализуйте функцию get_ttest_strat_pvalue


In [15]:
import numpy as np
import pandas as pd
from scipy import stats


# Вычисляет стратифицированное среднее
def calculate_stratified_mean(df, weights):
    # вычисляю среднее для каждой страты
    strat_mean = df.groupby('strat')['metric'].mean()
    
    # веса х страты
    df_means_weights = pd.merge(
        strat_mean,
        pd.Series(weights, name='weight'),
        how='inner',
        left_index=True,
        right_index=True
    )
    
    # пересчет весов по данным внутри выборки - расхождение с условием
    # если в данных df не было каких-то страт, то часть weights затрётся и сумма весов будет меньше 1
    df_means_weights['weight'] = df_means_weights['weight'] / df_means_weights['weight'].sum()
    
    # стратифицированное среднее = среднее страты * вес страты (в данном случае внутри выборки)
    mean_strat = (df_means_weights['weight'] * df_means_weights['metric']).sum()
    
    return mean_strat


# Вычисляет стратифицированную дисперсию
def calculate_strat_var(df, weights):
    
    # вычисляю дисперсию по стратам
    strat_vars = df.groupby('strat')['metric'].var()
    
    # дисперсия по стратам х веса
    df_vars_weights = pd.merge(
        strat_vars,
        pd.Series(weights, name='weight'),
        how='inner',
        left_index=True,
        right_index=True
    )
    
    # пересчет весов по данным внутри выборки - расхождение с условием
    # если в данных df не было каких-то страт, то часть weights затрётся и сумма весов будет меньше 1
    df_vars_weights['weight'] = df_vars_weights['weight'] / df_vars_weights['weight'].sum()
    
    # стратифицированная дисперсия = дисперсия страты * вес страты (в данном случае внутри выборки)
    var_strat = (df_vars_weights['weight'] * df_vars_weights['metric']).sum()
    return var_strat


def check_stratified_test(df_control, df_pilot, weights):

    # считаю стратифицированное среднее по выборкам
    mean_strat_control = calculate_stratified_mean(df_control, weights)
    mean_strat_pilot = calculate_stratified_mean(df_pilot, weights)
    
    # считаю стратифицированную дисперсию
    var_strat_control = calculate_strat_var(df_control, weights)
    var_strat_pilot = calculate_strat_var(df_pilot, weights)
    
    # числитель ттеста
    delta_mean_strat = mean_strat_pilot - mean_strat_control
    
    # знаменатель ттеста
    std_mean_strat = (var_strat_pilot / len(df_pilot) + var_strat_control / len(df_control)) ** 0.5
    
    # ттест
    t = delta_mean_strat / std_mean_strat
    
    # np.abs(t) возвращает модуль ттеста
    # stats.norm.cdf считает площадь под нормированным графиком распределения t-значения
    pvalue = (1 - stats.norm.cdf(np.abs(t))) * 2
    
    return pvalue


def get_ttest_strat_pvalue(metrics_strat_a_group, metrics_strat_b_group):
    """
    Применяет постстратификацию, возвращает pvalue.

    Веса страт считаем по данным обеих групп.
    Предполагаем, что эксперимент проводится на всей популяции.
    Веса страт нужно считать по данным всей популяции.

    :param metrics_strat_a_group (np.ndarray): значения метрик и страт группы A.
        shape = (n, 2), первый столбец - метрики, второй столбец - страты.
    :param metrics_strat_b_group (np.ndarray): значения метрик и страт группы B.
        shape = (n, 2), первый столбец - метрики, второй столбец - страты.
        
    :return (float): значение p-value
    """

    # считаем веса каждой страты - типа они одинаковые для каждой группы и посчитаны по всей популяции
    one_strat = np.count_nonzero(metrics_strat_a_group[:, 1] == 1) + np.count_nonzero(metrics_strat_b_group[:, 1] == 1)   
    zero_strat = len(metrics_strat_a_group) + len(metrics_strat_b_group) - one_strat
    all_strats = one_strat + zero_strat
    strats_dict = {
        1: one_strat / all_strats,
        0: zero_strat / all_strats
    }  

    df_control = pd.DataFrame(metrics_strat_a_group, columns = ['metric', 'strat'])
    df_pilot   = pd.DataFrame(metrics_strat_b_group, columns = ['metric', 'strat'])
    pvalue = check_stratified_test(df_control, df_pilot, strats_dict)
    
    return pvalue


In [16]:
metrics_strat_a_group = np.array([
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
]).T

metrics_strat_b_group = np.array([
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
]).T

print(get_ttest_strat_pvalue(metrics_strat_a_group, metrics_strat_b_group))
# pvalue = 0.037056

0.037056218564119
